# PTT 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例 ，完成以下問題：

* ① 印出最新文章的「作者」「標題」「時間」
* ② 印出第一頁所有文章的「作者」「標題」「時間」
* ③ 試著爬爬看其他版的文章


In [10]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Stock/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")
soup

<!DOCTYPE html>
<html><head>
		<meta charset="utf-8"/>
		

<meta content="width=device-width, initial-scale=1" name="viewport"/>

<title>看板 Stock 文章列表 - 批踢踢實業坊</title>

<link href="//images.ptt.cc/bbs/v2.27/bbs-common.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-base.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-custom.css" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/pushstream.css" media="screen" rel="stylesheet" type="text/css"/>
<link href="//images.ptt.cc/bbs/v2.27/bbs-print.css" media="print" rel="stylesheet" type="text/css"/>




	</head>
    <body>
		
<div id="topbar-container">
	<div class="bbs-content" id="topbar">
		<a href="/bbs/" id="logo">批踢踢實業坊</a>
		<span>›</span>
		<a class="board" href="/bbs/Stock/index.html"><span class="board-label">看板 </span>Stock</a>
		<a class="right small" href="/about.html">關於我們</a>
		<a class="right small" href="/contact.html">聯絡

### ② 印出第一頁所有文章的「作者」「標題」「時間」

In [11]:
# 方法一：只從首頁
for d in soup.find_all(class_="r-ent"):
    print('標題： ', d.find(class_='title').text.replace('\t', '').replace('\n', ''))
    print('作者： ', d.find(class_='author').text.replace('\t', '').replace('\n', ''))
    print('時間： ', d.find(class_='date').text.replace('\t', '').replace('\n', ''))
    

標題：  [公告] gR7P4zXH 水桶一個月
作者：  eyespot
時間：   3/15
標題：  [公告] monkeyking18 水桶一週
作者：  eyespot
時間：   3/15
標題：  [心得] 想抄底的別做白工了
作者：  Bruce003
時間：   3/15
標題：  [公告] 精華區導覽Q&A
作者：  IanLi
時間：   1/25
標題：  [公告] Stock 板規V2.6             (2019/06/20)
作者：  eyespot
時間：  11/03
標題：  [公告] 關於武漢肺炎新聞規範
作者：  noldorelf
時間：   1/25
標題：  [閒聊] 2020/03/13 盤後閒聊
作者：  wickwolf
時間：   3/13


In [12]:
# 方法二：進入內頁抓
for d in soup.find_all(class_="title"):
    try:
        print('標題： ', d.text.replace('\t', '').replace('\n', ''))
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        print('作者： ' + r.find(class_='article-meta-value').text)
        print('時間： ' + r.find_all(class_='article-meta-value')[-1].text)
    except:
        continue

標題：  [公告] gR7P4zXH 水桶一個月
作者： eyespot (追求內心的自在)
時間： Sun Mar 15 22:57:56 2020
標題：  [公告] monkeyking18 水桶一週
作者： eyespot (追求內心的自在)
時間： Sun Mar 15 23:01:36 2020
標題：  [心得] 想抄底的別做白工了
作者： Bruce003 (Bruce Chen)
時間： Sun Mar 15 23:04:13 2020
標題：  [公告] 精華區導覽Q&A
作者： IanLi (IanLi)
時間： Sun Jan 25 23:18:20 2015
標題：  [公告] Stock 板規V2.6             (2019/06/20)
標題：  [公告] 關於武漢肺炎新聞規範
作者： noldorelf (屏東周渝民)
時間： Sat Jan 25 15:02:33 2020
標題：  [閒聊] 2020/03/13 盤後閒聊
作者： wickwolf (呆狼)
時間： Fri Mar 13 14:00:00 2020


### ① 印出最新文章的「作者」「標題」「時間」

In [13]:
# 因為原始的資料難以判斷「新/舊」，因此我們必須進入內頁抓取詳細的時間，最終存成 List of Dict 的形式再自行排序。

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
# 補充：List of Dict 的排序方法
# https://stackoverflow.com/questions/72899/how-do-i-sort-a-list-of-dictionaries-by-a-value-of-the-dictionary
posts[-1]

{'標題': '[心得] 想抄底的別做白工了',
 '作者': 'Bruce003 (Bruce Chen)',
 '時間': 'Sun Mar 15 23:04:13 2020'}

### ③ 試著爬爬看其他版的文章

In [14]:
# 改成 Lifeismoney 版，抓出最新一筆的文章

import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Lifeismoney/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

{'標題': '[公告] 違規檢舉區', '作者': 'medama ( )', '時間': 'Thu Dec 19 22:54:37 2019'}

In [16]:
# 改成 Gossiping 版，發生錯誤，因為八卦版會跳轉到一個「十八歲的同意驗證頁面」導致錯誤。


import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url)

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href']).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

IndexError: list index out of range

In [17]:
# 改成 Gossiping 版，發生錯誤，因為八卦版會跳轉到一個「十八歲的同意驗證頁面」導致錯誤。
# 參考圖片下載時的解法，加上 cookies 繞過驗證（後面課程會再進行補充）


import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url, cookies={'over18': '1'})

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href'], cookies={'over18': '1'}).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

{'標題': '[公告] 八卦板板規(2019.08.21)',
 '作者': 'arsonlolita (蘿莉塔)',
 '時間': 'Wed Aug 21 08:33:39 2019'}

In [33]:
# 我們發現這個時間好像不是正確的，原因是我們前面存到的時間，其實不是正確的格式

# 改成 Gossiping 版，發生錯誤，因為八卦版會跳轉到一個「十八歲的同意驗證頁面」導致錯誤。
# 參考圖片下載時的解法，加上 cookies 繞過驗證（後面課程會再進行補充）


import requests
from bs4 import BeautifulSoup

url = 'https://www.ptt.cc/bbs/Gossiping/index.html'
r = requests.get(url, cookies={'over18': '1'})

soup = BeautifulSoup(r.text, "html5lib")

posts = []

for d in soup.find_all(class_="title"):
    try:
        post = {}
        post['標題'] = d.text.replace('\t', '').replace('\n', '')
        
        r = BeautifulSoup(requests.get('https://www.ptt.cc'+d.find('a')['href'], cookies={'over18': '1'}).text, "html5lib")
        post['作者'] = r.find(class_='article-meta-value').text
        post['時間'] = r.find_all(class_='article-meta-value')[-1].text
        post['時間'] = datetime.strptime(post['時間'], "%a %b %d %H:%M:%S %Y")
        # 時間轉換：https://stackoverflow.com/questions/10256093/how-to-convert-ctime-to-datetime-in-python
        
        posts.append(post)
    except:
        continue
        
posts = sorted(posts, key= lambda x: x['時間'])
posts[-1]

{'標題': '[新聞] 共青團捧網紅 網友：她怎麼上YouTube的',
 '作者': 'KahoJyumonji (Kahojyumonji)',
 '時間': datetime.datetime(2019, 12, 12, 11, 16, 52)}